In [1]:
import os 
import csv
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
import json

In [15]:
o = pd.DataFrame()
for file in os.listdir('../ce_data/StatCache'):
    x = pd.read_csv("../ce_data/StatCache/"+file)
    x['id'] = x.index
    x['id'] = x['id'].apply(lambda x: file[:-4] + "_" + str(x))
    o = pd.concat([x, o], ignore_index=True)
o = o.sort_values("id", ascending=True, ignore_index=True).fillna(0)
print(o)
for i in ['32','64','128','256','512','1024','2048','4096','8192']:
    o['lru_'+i] = o['lru_'+i]/o['total_accesses']
print(o)

        random_32  lru_32  random_64  lru_64  random_128  lru_128  random_256  \
0        0.000000      16   0.000000       0    0.000000        0    0.000000   
1        0.000000      32   0.000000       0    0.000000        0    0.000000   
2        0.000000       0   0.000000       0    0.000000        0    0.000000   
3        0.000619     240   0.000497     240    0.000436      240    0.000436   
4        0.001229     416   0.000985     416    0.000924      416    0.000863   
...           ...     ...        ...     ...         ...      ...         ...   
141556   0.000000       0   0.000000       0    0.000000        0    0.000000   
141557   0.000000       0   0.000000       0    0.000000        0    0.000000   
141558   0.000000       0   0.000000       0    0.000000        0    0.000000   
141559   0.000000       0   0.000000       0    0.000000        0    0.000000   
141560   0.000000       0   0.000000       0    0.000000        0    0.000000   

        lru_256  random_512

In [16]:
print(o['lru_32'][0])

3.158958807177154e-05


In [10]:
cache_misses = pd.read_csv("../ce_data/caches/llc_miss_ratio.csv")
cache_misses = cache_misses.sort_values("id", ignore_index=True)
print(cache_misses)

                           id        32        64       128       256  \
0         400.perlbench-41B_0  1.000000  1.000000  1.000000  0.998652   
1         400.perlbench-41B_1  1.000000  1.000000  1.000000  1.000000   
2        400.perlbench-41B_10  1.000000  0.997375  0.997375  0.997375   
3       400.perlbench-41B_100  0.996656  0.996656  0.996656  0.993311   
4       400.perlbench-41B_101  1.000000  1.000000  1.000000  1.000000   
...                       ...       ...       ...       ...       ...   
141556        657.xz_s-56B_95  1.000000  1.000000  1.000000  1.000000   
141557        657.xz_s-56B_96  1.000000  1.000000  1.000000  1.000000   
141558        657.xz_s-56B_97  1.000000  1.000000  1.000000  1.000000   
141559        657.xz_s-56B_98  1.000000  1.000000  1.000000  1.000000   
141560        657.xz_s-56B_99  1.000000  1.000000  1.000000  1.000000   

             512      1024      2048      4096      8192  
0       0.889488  0.828841  0.803235  0.773585  0.768194  
1    

In [11]:
# Little check
d = {}
for i,j in zip(cache_misses["id"], o["id"]):
    if i != j:
        print(i)
    if i in d: 
        print(i)
    d[i]=1

In [12]:
from sklearn.metrics import mean_squared_error
for i in ['32','64','128','256','512','1024','2048','4096','8192']:
    c_acc_lru = mean_squared_error((o['lru_'+i] / o['lru_512']).replace(np.inf, 0).fillna(0), cache_misses[i])
    c_acc_rand = mean_squared_error(o['random_'+i], cache_misses[i])
    print(f'Cache size {i}\t: lru_mse = {c_acc_lru},\t random_mse = {c_acc_rand}')

Cache size 32	: lru_mse = 15686.843828412379,	 random_mse = 0.7688156845581666
Cache size 64	: lru_mse = 7533.084105267284,	 random_mse = 0.773865207083091
Cache size 128	: lru_mse = 4589.306667769615,	 random_mse = 0.7683378564248594
Cache size 256	: lru_mse = 2293.9706553855126,	 random_mse = 0.746738916300773
Cache size 512	: lru_mse = 0.19365307801926943,	 random_mse = 0.6474104323601267
Cache size 1024	: lru_mse = 0.1867318782536081,	 random_mse = 0.3850742052080373
Cache size 2048	: lru_mse = 0.1918957771407773,	 random_mse = 0.33225249039596066
Cache size 4096	: lru_mse = 0.18917798076323786,	 random_mse = 0.274371042118911
Cache size 8192	: lru_mse = 0.16951024831761782,	 random_mse = 0.22390588890801244


In [13]:
accesses_l1d = pd.read_csv("../ce_data/caches/l1d_accesses.csv")
accesses_l1d = accesses_l1d.sort_values("id", ignore_index=True)
accesses_llc = pd.read_csv("../ce_data/caches/llc_accesses.csv")
accesses_llc = accesses_llc.sort_values("id", ignore_index=True)
print(accesses_llc)

                           id   32   64  128  256  512  1024  2048  4096  8192
0         400.perlbench-41B_0  742  742  742  742  742   742   742   742   742
1         400.perlbench-41B_1  911  911  911  911  911   911   911   911   911
2        400.perlbench-41B_10  381  381  381  381  381   381   381   381   381
3       400.perlbench-41B_100  299  299  299  299  299   299   299   299   299
4       400.perlbench-41B_101  370  370  370  370  370   370   370   370   370
...                       ...  ...  ...  ...  ...  ...   ...   ...   ...   ...
141556        657.xz_s-56B_95  838  838  838  838  838   838   838   838   838
141557        657.xz_s-56B_96  836  836  836  836  836   836   836   836   836
141558        657.xz_s-56B_97  836  836  836  836  836   836   836   836   836
141559        657.xz_s-56B_98  838  838  838  838  838   838   838   838   838
141560        657.xz_s-56B_99  836  836  836  836  836   836   836   836   836

[141561 rows x 10 columns]


In [14]:
from sklearn.metrics import mean_squared_error
for i in ['1024','2048','4096','8192']:
    tmp = o['lru_'+i] * accesses_l1d[i]/accesses_llc[i]
    c_acc_lru = mean_squared_error(tmp.replace(np.inf, 0).fillna(0) , cache_misses[i])
    c_acc_rand = mean_squared_error(o['random_'+i], cache_misses[i])
    print(f'Cache size {i}\t: lru_mse = {c_acc_lru},\t random_mse = {c_acc_rand}')

Cache size 1024	: lru_mse = 0.3646853995341936,	 random_mse = 0.3850742052080373
Cache size 2048	: lru_mse = 0.20688329727851754,	 random_mse = 0.33225249039596066
Cache size 4096	: lru_mse = 0.17498801306984907,	 random_mse = 0.274371042118911
Cache size 8192	: lru_mse = 0.15722359927395654,	 random_mse = 0.22390588890801244
